In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 31.8 MB/s eta 0:00:00


In [2]:
import pennylane as qml
import numpy as np

N_QUBITS = 2
NOISE_STRENGTH = 0.05
NUM_LAYERS = 3
NUM_STEPS = 500  # Increased number of steps
STEP_SIZE = 0.1  # Adjusted step size

dev = qml.device("default.mixed", wires=N_QUBITS, shots=1000)

@qml.qnode(dev)
def noisy_circuit(var_params, x=None):
    for i, xi in enumerate(x):
        qml.RX(xi, wires=i)
        qml.DepolarizingChannel(NOISE_STRENGTH, wires=i)

    for _ in range(NUM_LAYERS):
        for i in range(N_QUBITS):
            qml.Rot(var_params[i], var_params[i + N_QUBITS], var_params[i + 2 * N_QUBITS], wires=i)
        for i in range(N_QUBITS - 1):
            qml.CNOT(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(N_QUBITS)]

def noisy_cost(var_params, x=None, y=None):
    predictions = np.array(noisy_circuit(var_params, x=x))
    return np.sum((predictions - y) ** 2)

def optimize_with_noise():
    # Adjusting the initial parameters with a smaller range
    var_params = qml.numpy.random.uniform(low=-0.1, high=0.1, size=3 * N_QUBITS)
    true_thetas = np.pi * np.random.rand(N_QUBITS)

    initial_params = var_params.copy()

    # Using the Adam optimizer with an increased step size
    opt = qml.AdamOptimizer(stepsize=STEP_SIZE)

    for i in range(NUM_STEPS):
        # Evaluate the initial circuit with true parameters
        observed_values = np.array(noisy_circuit(var_params, x=true_thetas))

        # Pass true_thetas as the target values for the optimization
        var_params = opt.step(noisy_cost, var_params, x=true_thetas, y=true_thetas)

        # Print the cost every 50 steps
        if i % 50 == 0:
            print(f"Step {i+1}/{NUM_STEPS} - Cost: {noisy_cost(var_params, x=true_thetas, y=true_thetas)}")

    print("\nOptimization complete.")
    print("Initial parameters:", initial_params)
    print("Optimized parameters with noise:", var_params)
    print("Final cost with noise:", noisy_cost(var_params, x=true_thetas, y=true_thetas))

if __name__ == "__main__":
    optimize_with_noise()

Step 1/500 - Cost: 1.8195191723716828
Step 51/500 - Cost: 0.3155386353508649
Step 101/500 - Cost: 0.317684422890525
Step 151/500 - Cost: 0.32437010547881706
Step 201/500 - Cost: 0.3255886652333118
Step 251/500 - Cost: 0.3212884225069231
Step 301/500 - Cost: 0.31456209998000745
Step 351/500 - Cost: 0.31900548736622664
Step 401/500 - Cost: 0.3266438930184771
Step 451/500 - Cost: 0.3206639702614002

Optimization complete.
Initial parameters: [-0.00285521 -0.07327956  0.00541612  0.03572836 -0.02953984  0.09527576]
Optimized parameters with noise: [ 0.41875364  0.22376084  1.7102812   0.24668548 -2.06254397 -0.30670157]
Final cost with noise: 0.32848853334070277
